# A Step-by-Step Guide to HoloFusion example part 2/4 
  # (accu implementation)


# Setup
Firstly, we import all the module from Holofusion that we will use.

In [1]:
from holofusion import HoloFusion, HoloFusionSession
from time import time as t

##   Initialization
In this part, we create the HoloFusion and Session object that we will use for this example.

In [2]:
        holo_obj = HoloFusion(algorithm=2, training_data=0, multiple_weights=0)
        session = HoloFusionSession("Session", holo_obj)
        fx = open('execution_time.txt', 'w')
        

/home/gmichalo/anaconda/envs/python27/lib/python2.7/site-packages/sqlalchemy/dialects/mysql/base.py:1576: Warning: '@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead
  cursor.execute('SELECT @@tx_isolation')


## Read Input from file
Test data will be read using the Session's ingestor.
After ingesting the test data will be loaded into MySQL tables along with entries in the a metadata table.

In [3]:
        list_time = []
        start_time = t()

        session.ingest_dataset("data/clean_flight/flight-data-test.csv")
        d = t()-start_time
        list_time.append(d)
        fx.write('ingest csv time: '+str(d)+'\n')
        print 'ingest csv time: '+str(d)+'\n'
        
        start_time = t()
        session.adding_training_data("data/clean_flight/flight-data_training.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('adding training data: ' + str(d) + '\n')
        print 'adding training time: ' + str(d) + '\n'
        
        print 'Clean table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_clean", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()
        print 'Don\'t know table'
        sql = holo_obj.dataengine.get_table_to_dataframe("C_dk", session.dataset)
        sql.select('Source','Flight_Num','Scheduled_Dept','Actual_Dept').show()

ingest csv time: 9.2502450943

adding training time: 2.1162788868

Clean table
+------+----------+--------------+-----------+
|Source|Flight_Num|Scheduled_Dept|Actual_Dept|
+------+----------+--------------+-----------+
+------+----------+--------------+-----------+

Don't know table
+-------------------+--------------------+--------------+--------------------+
|             Source|          Flight_Num|Scheduled_Dept|         Actual_Dept|
+-------------------+--------------------+--------------+--------------------+
|                 aa|2011-12-01-AA-100...|          1:55|                2:07|
|     flightexplorer|2011-12-01-AA-100...|          null|                2:07|
|    airtravelcenter|2011-12-01-AA-100...|          null|                5:00|
|         myrateplan|2011-12-01-AA-100...|          null|                5:00|
|        helloflight|2011-12-01-AA-100...|          null|                5:00|
|          flytecomm|2011-12-01-AA-100...|          null|                5:00|
|   

## Featurization
We create the appropriate dictionary to run accu algorithm

In [4]:
        start_time = t()
        session.feature()
        d = t() - start_time
        list_time.append(d)
        fx.write('creating feature table time: '+str(d)+'\n')

        print ' feature table time: '+str(d)+'\n'

 feature table time: 2.91080880165



#  Learning
In the learning phase, we use the accu algorithm to get the most appropriate value for each object


In [5]:
        start_time = t()
        session.inference(iterations = 100)
        d = t() - start_time
        list_time.append(d)
        fx.write('inference time: ' + str(d) + '\n')
        print 'inference time: ' + str(d) + '\n'
        

Print weight_accuracy for the first 10 sources:
aa 0.99
helloflight 0.768596702855
boston 0.999593330622
weather 0.567477284874
airtravelcenter 0.691769876149
flightview 0.751453488372
flightstats 0.284258608573
usatoday 0.682714617169
ord 0.461244541485
panynj 0.780718140307
inference time: 22.2446730137



# Evaluation
 In this part, we find the accuracy of our results by comparing them to ground truth for our data

In [6]:
        start_time = t()
        session.accuracy("data/clean_flight/flight-data_truth.csv")
        d = t() - start_time
        list_time.append(d)
        fx.write('time to calculate accuracy: ' + str(d) + '\n')
        print 'time to calculate accuracy: ' + str(d) + '\n'
        fx.close()

['Flight_Num', 'Scheduled_Dept', 'Actual_Dept', 'Scheduled_Arrival', 'Actual_Arrival']
show the final table
+--------------------+-----------------+------------+
|            rv_index|          rv_attr|assigned_val|
+--------------------+-----------------+------------+
|2011-12-07-UA-246...|Scheduled_Arrival|        1:27|
|2011-12-08-CO-443...|   Scheduled_Dept|        4:25|
|2011-12-17-UA-246...|   Scheduled_Dept|        null|
|2011-12-29-AA-610...|   Scheduled_Dept|        null|
|2012-01-01-AA-203...|      Actual_Dept|        2:22|
|2011-12-20-AA-290...|      Actual_Dept|        5:27|
|2011-12-16-AA-645...|Scheduled_Arrival|        9:20|
|2011-12-01-UA-126...|   Actual_Arrival|       12:56|
|2011-12-04-AA-188...|   Actual_Arrival|        1:47|
|2011-12-12-UA-858...|Scheduled_Arrival|        null|
|2011-12-27-AA-309...|Scheduled_Arrival|        null|
|2011-12-26-AA-166...|Scheduled_Arrival|       12:10|
|2011-12-13-CO-488...|   Scheduled_Dept|        null|
|2011-12-02-AA-129...|   Act